In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/euro_cup", True)




Out[40]: False

In [0]:
%sql
CREATE OR REPLACE TABLE euro_cup(
  grp VARCHAR(2),
  team VARCHAR(20)
)



In [0]:
%sql
INSERT INTO euro_cup(grp,team) VALUES("A", "Italy"),("A", "Wales"),("A", "Switzerland"),("A", "Turkey"),("B", "Belgium"),("B", "Denmark"),("B", "Finland"),("B", "Russia"),("C", "Netherlands"),("C", "Austria"),("C", "Ukraine"),("C", "North Macedonia"),("D", "England"),("D", "Croatia"),("D", "Czech Republic"),("D", "Scotland"),("E", "Sweden"),("E", "Spain"),("E", "Slovakia"),("E", "Poland"),("F", "France"),("F", "Germany"),("F", "Portugal"),("F", "Hungary");

num_affected_rows,num_inserted_rows
24,24


In [0]:
%sql
SELECT * FROM euro_cup

grp,team
A,Italy
A,Wales
A,Switzerland
A,Turkey
B,Belgium
B,Denmark
B,Finland
B,Russia
C,Netherlands
C,Austria


In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/euro_cup_group_stage", True)


Out[41]: False

In [0]:
%sql
CREATE OR REPLACE TABLE euro_cup_group_stage AS (
  WITH newCte AS (
  WITH cte AS (
  SELECT 
  CONCAT(a.team," vs ",b.team) AS Pld,
  a.team AS team1,a.grp AS grp1,
  b.team AS team2,b.grp AS grp2,
  floor(rand()*5) AS team1_goals,
  floor(rand()*5) AS team2_goals,
  CASE
    WHEN team1_goals > team2_goals THEN 3
    WHEN team1_goals = team2_goals THEN 1
    ELSE 0
  END AS team1pts,
  CASE
    WHEN team1_goals < team2_goals THEN 3
    WHEN team1_goals = team2_goals THEN 1
    ELSE 0
  END AS team2pts
  FROM
  euro_cup a 
  CROSS JOIN
  euro_cup b
  ON
  a.grp = b.grp 
  AND 
  a.team < b.team 
)
-- SELECT * FROM cte
SELECT team1,grp1,team1_goals,team2_goals,team1pts FROM cte 
UNION ALL
SELECT team2,grp2,team1_goals,team2_goals,team2pts FROM cte 
 
)
SELECT
  team1 AS team,grp1 AS Grp,3 AS Pld,
  FLOOR(SUM(team1pts)/3) AS w,
  CASE 
    WHEN SUM(team1pts) % 3 = 0 THEN 0
    ELSE 1
  END AS d,
  CASE 
    WHEN w + d = 3 THEN 0
    WHEN w + d = 2 THEN 1
    WHEN w + d = 1 THEN 2
    ELSE 3
  END AS l,
  SUM(team1_goals) AS gf,
  SUM(team2_goals) AS ga,
  SUM(team1_goals) - SUM(team2_goals) AS gd,
  SUM(team1pts) AS pts
FROM
  newCte
GROUP BY
team1,Grp
ORDER BY 
Grp,pts DESC
  
)


num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from euro_cup_group_stage

team,Grp,Pld,w,d,l,gf,ga,gd,pts
Wales,A,3,2,0,1,3,5,-2,6
Switzerland,A,3,1,0,2,3,9,-6,3
Italy,A,3,0,1,2,4,6,-2,1
Turkey,A,3,0,1,2,4,1,3,1
Finland,B,3,2,0,1,5,5,0,6
Denmark,B,3,1,1,1,6,5,1,5
Russia,B,3,1,1,1,8,9,-1,4
Belgium,B,3,1,0,2,5,6,-1,3
Austria,C,3,2,1,0,7,4,3,7
North Macedonia,C,3,1,1,1,8,4,4,4


In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/euro_cup_knockout", True)

Out[42]: False

In [0]:
%sql
CREATE OR REPLACE TABLE euro_cup_knockout as (
  WITH cte AS (
SELECT team,Grp,gd,pts,row_number() OVER (PARTITION BY Grp ORDER BY pts DESC) AS rank FROM euro_cup_group_stage
)

SELECT * FROM cte WHERE rank <= 3 
ORDER BY CASE WHEN rank <= 2 THEN pts END DESC, 
          CASE WHEN rank <= 2 THEN gd END DESC, 
          CASE WHEN rank = 3 THEN pts END DESC, 
          CASE WHEN rank = 3 THEN gd END DESC 
          LIMIT 16
)



num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from euro_cup_knockout

team,Grp,gd,pts,rank
Austria,C,3,7,1
Hungary,F,-3,7,1
Germany,F,7,6,2
England,D,4,6,2
Croatia,D,3,6,1
Finland,B,0,6,1
Wales,A,-2,6,1
Denmark,B,1,5,2
North Macedonia,C,4,4,2
Poland,E,0,4,1


Third-placed 
teams
qualify 
from groups	      
            1B	1C
                vs	1E
                    vs	1F
vs
A	B	C	D			3A	3D	3B	3C
A	B	C		E		3A	3E	3B	3C
A	B	C			F	3A	3F	3B	3C
A	B		D	E		3D	3E	3A	3B
A	B		D		F	3D	3F	3A	3B
A	B			E	F	3E	3F	3B	3A
A		C	D	E		3E	3D	3C	3A
A		C	D		F	3F	3D	3C	3A
A		C		E	F	3E	3F	3C	3A
A			D	E	F	3E	3F	3D	3A
B	C	D	E		  3E	3D	3B	3C
B	C	D		F	  3F	3D	3C	3B
B	C		E	F	  3F	3E	3C	3B
B		D	E	F	  3F	3E	3D	3B
C	D	E	F	    3F	3E	3D	3C


In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/third_rank_mapping", True)

Out[43]: False

In [0]:
%sql
CREATE OR REPLACE TABLE third_rank_mapping(
  match_pair STRING,
  1B_VS STRING,
  1C_VS STRING,
  1E_VS STRING,
  1F_VS STRING
)
 

 
 
-- SELECT * FROM third_rank_mapping

In [0]:
%sql
INSERT INTO third_rank_mapping( match_pair, 1B_VS, 1C_VS, 1E_VS, 1F_VS)
VALUES
('ABCD','A','D','B','C'),
('ABCE','A','E','B','C'),
('ABCF','A','F','B','C'),
('ABDE','D','E','A','B'),
('ABDF','D','F','A','B'),
('ABEF','E','F','B','A'),
('ACDE','E','D','C','A'),
('ACDF','F','D','C','A'),
('ACEF','E','F','C','A'),
('ADEF','E','F','D','A'),
('BCDE','E','D','B','C'),
('BCDF','F','D','C','B'),
('BCEF','F','E','C','B'),
('BDEF','F','E','D','B'),
('CDEF','F','E','D','C')

num_affected_rows,num_inserted_rows
15,15


In [0]:
%sql
SELECT concat_ws('',array_agg(Grp)) AS all_third_grp
FROM (
    SELECT Grp
    FROM euro_cup_knockout
    WHERE rank >= 3
    ORDER BY Grp ASC
) AS subquery
 

all_third_grp
BCDF


In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/final_mapping", True)

Out[44]: False

In [0]:
%sql
CREATE OR REPLACE TABLE final_mapping AS(
SELECT *
FROM third_rank_mapping
WHERE match_pair = (
    SELECT CONCAT_WS('', array_agg(Grp)) AS all_third_grp
    FROM (
        SELECT DISTINCT Grp
        FROM euro_cup_knockout
        WHERE rank >= 3
        ORDER BY Grp ASC
    ) AS subquery
)
)


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM final_mapping

match_pair,1B_VS,1C_VS,1E_VS,1F_VS
BCDF,F,D,C,B


In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/euro_cup_quarterfinal", True)

Out[45]: False

In [0]:
%sql
CREATE OR REPLACE TABLE euro_cup_quarterfinal AS (
WITH cte3 AS (
WITH cte2 AS (
WITH cte AS (
SELECT team FROM euro_cup_knockout WHERE rank = 1 AND Grp IN ('B') 
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 3 AND Grp IN (SELECT 1B_VS FROM final_mapping) 
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 1 AND Grp in ('A') 
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 2 AND Grp in ('C')
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 1 AND Grp in ('F')
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 3 AND Grp IN (SELECT 1F_VS FROM final_mapping)
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 2 AND Grp in ('D')
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 2 AND Grp in ('E')
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 1 AND Grp in ('E')
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 3 AND Grp IN (SELECT 1E_VS FROM final_mapping)
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 1 AND Grp in ('D')
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 2 AND Grp in ('F')
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 1 AND Grp in ('C')
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 3 AND Grp IN (SELECT 1C_VS FROM final_mapping)
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 2 AND Grp in ('A')
UNION
SELECT team FROM euro_cup_knockout WHERE rank = 2 AND Grp in ('B')
)
SELECT row_number() OVER (ORDER BY(SELECT NULL)) AS Pos,* FROM cte
)
SELECT t1.team AS team1,t2.team AS team2,
case when ROUND(RAND()) = 0 then "loss" else "win" end AS team1_result,
case when team1_result = "loss" then "win" else "loss" end as team2_result
FROM cte2 t1 JOIN cte2 t2 
ON t1.Pos = t2.pos - 1 
WHERE t1.pos % 2 != 0
)
SELECT
CASE
  WHEN team1_result = 'win' THEN team1
  ELSE team2
  END AS winner
FROM cte3
)

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM euro_cup_quarterfinal

winner
France
Wales
Hungary
England
Netherlands
Croatia
Czech Republic
Denmark


In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/euro_cup_semifinal", True)

Out[46]: False

In [0]:
%sql
CREATE OR REPLACE TABLE euro_cup_semifinal(
WITH cte2 AS (
WITH cte AS (
SELECT row_number() OVER (ORDER BY(SELECT NULL)) AS Pos,* FROM euro_cup_quarterfinal
)
SELECT t1.winner AS team1,t2.winner AS team2,
case when ROUND(RAND()) = 0 then "loss" else "win" end AS team1_result,
case when team1_result = "loss" then "win" else "loss" end as team2_result
FROM cte t1 JOIN cte t2 
ON t1.Pos = t2.pos - 1 
WHERE t1.pos % 2 != 0
)
SELECT
CASE
  WHEN team1_result = 'win' THEN team1
  ELSE team2
  END AS winner
FROM cte2
)

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM euro_cup_semifinal

winner
Wales
Hungary
Netherlands
Czech Republic


In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/euro_cup_final", True)

Out[47]: False

In [0]:
%sql
CREATE OR REPLACE TABLE euro_cup_final AS(
WITH cte2 AS (
WITH cte AS (
SELECT row_number() OVER (ORDER BY(SELECT NULL)) AS Pos,* FROM euro_cup_semifinal
)
SELECT t1.winner AS team1,t2.winner AS team2,
case when ROUND(RAND()) = 0 then "loss" else "win" end AS team1_result,
case when team1_result = "loss" then "win" else "loss" end as team2_result
FROM cte t1 JOIN cte t2 
ON t1.Pos = t2.pos - 1 
WHERE t1.pos % 2 != 0
)
SELECT
CASE
  WHEN team1_result = 'win' THEN team1
  ELSE team2
  END AS winner
FROM cte2
)

num_affected_rows,num_inserted_rows


In [0]:
%sql

SELECT * FROM euro_cup_final


winner
Wales
Czech Republic


In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/euro_cup_winner", True)

Out[48]: False

In [0]:
%sql
CREATE OR REPLACE TABLE euro_cup_winner AS(
WITH cte2 AS (
WITH cte AS (
SELECT row_number() OVER (ORDER BY(SELECT NULL)) AS Pos,* FROM euro_cup_final
)
SELECT t1.winner AS team1,t2.winner AS team2,
case when ROUND(RAND()) = 0 then "loss" else "win" end AS team1_result,
case when team1_result = "loss" then "win" else "loss" end as team2_result
FROM cte t1 JOIN cte t2 
ON t1.Pos = t2.pos - 1 
WHERE t1.pos % 2 != 0
)
SELECT
CASE
  WHEN team1_result = 'win' THEN team1
  ELSE team2
  END AS winner
FROM cte2
)

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM euro_cup_winner

winner
Wales
